In [1]:
import signac
import numpy as np
import matplotlib.pyplot as plt
import freud
import gsd
import hoomd
import gsd.pygsd
import gsd.hoomd

In [5]:
project = signac.get_project("workspace")
for job in project:
    print(job)

fe36ee4052a01cae041ef5988477de10


In [8]:
state_dict = {"size":"medium", "process":"quench"}

job_list = project.find_jobs(state_dict)

In [9]:
for job in job_list:
    msd_path = job.fn("trajectory.gsd")
    print (msd_path)
    create_rdf(msd_path, atom_types=['f'], start = 95)

/Users/emilyepstein/Projects/ptb7-itic/workspace/fe36ee4052a01cae041ef5988477de10/trajectory.gsd


In [2]:
def atom_type_pos(frame, atom_types): 
    positions = []
    for idx, type_id in enumerate(frame.particles.typeid):
        if frame.particles.types[type_id] in atom_types:
            positions.append(frame.particles.position[idx])
    return positions

In [3]:
def create_rdf(job_list,
               atom_types='all',
               r_max = None,
               r_min = 0.1,
               nbins = 50,
               start = 100):
    
    gsd_file = msd_path ## maybe??
    
    f = gsd.pygsd.GSDFile(open(gsd_file, "rb"))
    trajectory = gsd.hoomd.HOOMDTrajectory(f) 
    if r_max is None:
        r_max = max(trajectory[-1].configuration.box[:3]) * 0.45 
    freud_rdf = freud.density.RDF(bins=nbins, r_max=r_max, r_min=r_min)
    for frame in trajectory[start:]:
        if atom_types == 'all':
            freud_rdf.compute(system=frame, reset=False)
        else:
            query_points = atom_type_pos(frame, atom_types)
            box = frame.configuration.box
            freud_rdf.compute(system=(box, query_points), reset = False)
        
 
    x = freud_rdf.bin_centers
    y = freud_rdf.rdf
    filename = '{}-trajectory.txt'.format(atom_types[0])
    
    np.savetxt(job.fn(filename), np.transpose([x,y]), delimiter=',', header= "bin_centers, rdf")
   
    f.close()
    return freud_rdf
